In [1]:
%set_env DBIKE_DEV=True
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error
from web.db_utils import get_stations, get_historical_weather, get_historical_availabilities
import datetime

env: DBIKE_DEV=True
True
2024-04-04 13:16:18,797 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-04-04 13:16:18,798 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-04 13:16:19,173 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-04-04 13:16:19,174 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-04 13:16:19,361 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-04-04 13:16:19,362 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-04 13:16:19,743 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-04 13:16:19,744 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `Station`
2024-04-04 13:16:19,744 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-04 13:16:19,840 INFO sqlalchemy.engine.Engine ROLLBACK
2024-04-04 13:16:20,126 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-04 13:16:20,126 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `CurrentWeather`
2024-04-04 13:16:20,127 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-04 13:16:20,223 INFO sqlal

In [6]:
DAYS_BACK = 30
FEATURE_COLUMNS = [
        "FeelsLike",
        "Humidity",
        "Pressure",
        "Temperature",
        "hour",
        "minutes",
        "month",
        "dayofweek",
        "date",
    ]
TARGET_COLUMNS = ["StandsAvailable"]

In [3]:
# Load weather
weather = get_historical_weather(DAYS_BACK)
print(f"Got {DAYS_BACK} days of data. ({len(weather)} rows)")
weather_df = pd.DataFrame(weather)
weather_df['DateTime'] = weather_df['DateTime'].dt.round('h').astype(int) // 10**9
weather_df.head(5)

2024-04-04 13:16:21,562 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-04 13:16:21,563 INFO sqlalchemy.engine.Engine SELECT `CurrentWeather`.`DateTime`, `CurrentWeather`.`FeelsLike`, `CurrentWeather`.`Humidity`, `CurrentWeather`.`Pressure`, `CurrentWeather`.`Sunrise`, `CurrentWeather`.`Sunset`, `CurrentWeather`.`Temperature`, `CurrentWeather`.`UVI`, `CurrentWeather`.`WeatherId`, `CurrentWeather`.`WindGust`, `CurrentWeather`.`WindSpeed`, `CurrentWeather`.`Rain1h`, `CurrentWeather`.`Snow1h` 
FROM `CurrentWeather` 
WHERE DATE(`CurrentWeather`.`DateTime`) >= %(DATE_1)s
2024-04-04 13:16:21,564 INFO sqlalchemy.engine.Engine [generated in 0.09451s] {'DATE_1': datetime.datetime(2024, 3, 5, 13, 16, 21, 467682)}
2024-04-04 13:16:21,852 INFO sqlalchemy.engine.Engine ROLLBACK
Got 30 days of data. (680 rows)


,DateTime,FeelsLike,Humidity,Pressure,Sunrise,Sunset,Temperature,UVI,WeatherId,WindGust,WindSpeed,Rain1h,Snow1h
0,1709798400,3.64,85,1019,2024-03-07 06:58:06,2024-03-07 18:14:13,7.06,None,803,None,5.66,None,None
1,1709798400,3.80,85,1019,2024-03-07 06:58:06,2024-03-07 18:14:14,7.19,None,803,None,5.66,None,None
2,1709798400,4.20,85,1019,2024-03-07 06:58:05,2024-03-07 18:14:13,7.34,None,803,None,5.14,None,None
3,1709802000,4.29,84,1020,2024-03-07 06:58:05,2024-03-07 18:14:13,7.48,None,803,None,5.36,None,None
4,1709805600,4.36,82,1020,2024-03-07 06:58:05,2024-03-07 18:14:13,7.92,None,803,None,6.71,None,None


In [4]:
# Helper function - get closest availability to the hour and round
def get_availability_on_the_hour(availabilities: list):
    def to_date(date_str):
        try:
            dt = datetime.datetime.strptime(date_str, '%Y-%m-%dT%H:%M:%SZ')
            return dt
        except:
            dt = datetime.datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
            return dt

    # current_hour = to_date(availabilities[0]["LastUpdated"]).hour

    availability_to_clean = availabilities[0]
    cleaned_availabilities = []

    current_hour = to_date(availabilities[0]["LastUpdated"]).hour

    for a in availabilities:
        t = to_date(a["LastUpdated"]) # Get the last updated time
        # print(t.hour, current_hour)
        if t.hour != current_hour:
            # New hour
            availability_to_clean["LastUpdated"] = to_date(availability_to_clean["LastUpdated"])
            cleaned_availabilities.append(availability_to_clean)
            current_hour = t.hour
            availability_to_clean = a
    return cleaned_availabilities

In [12]:
stations = get_stations()
station_ids = [s["Id"] for s in stations]
dfs = []
for id in station_ids:
    print(f"Training for station {id}")
    availability = get_historical_availabilities(id, DAYS_BACK)
    print(f"Got availability for {DAYS_BACK} days back ({len(availability)} rows)")
    availability_on_the_hour = get_availability_on_the_hour(availability)
    print(f"After collapsing time: {len(availability_on_the_hour)} rows")
    availability_df = pd.DataFrame(availability_on_the_hour)

    availability_df["hour"] = availability_df["LastUpdated"].dt.hour
    availability_df["minutes"] = availability_df["LastUpdated"].dt.minute
    availability_df["month"] = availability_df["LastUpdated"].dt.month
    availability_df["dayofweek"] = availability_df["LastUpdated"].dt.dayofweek
    availability_df["date"] = availability_df["LastUpdated"].dt.day
    availability_df['rounded_LastUpdated'] = pd.to_datetime(availability_df['LastUpdated']).dt.round('H')
    weather_df['rounded_DateTime'] = weather_df['DateTime'].dt.round('H')
    print(availability_df.head(5))
    print(weather_df.head(5))
    merged_data = pd.merge(availability_df, weather_df, left_on='rounded_LastUpdated', right_on="rounded_DateTime", how='inner')
    # print(len(merged_data))
    linear_model = LinearRegression()
    X_train, X_test, y_train, y_test = train_test_split(merged_data[FEATURE_COLUMNS], merged_data[TARGET_COLUMNS])
    linear_model.fit(X_train, y_train)

    
    print("===== EVALUATION =======")
    print(X_test)
    print(y_test)
    ynew = linear_model.predict(X_test)
    print(ynew)
    break

    with open(f'models/linear-regression/{id}.pkl', 'wb') as handle:
        pickle.dump(linear_model, handle, pickle.HIGHEST_PROTOCOL)
        print(f"pickle generated for linear model {id}")


2024-04-04 13:23:49,203 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-04 13:23:49,203 INFO sqlalchemy.engine.Engine SELECT `Station`.`Id`, `Station`.`Name`, `Station`.`PositionLatitude`, `Station`.`PositionLongitude`, `Station`.`Address`, `Station`.`City`, `Station`.`AcceptsCard`, `Station`.`TotalStands` 
FROM `Station`
2024-04-04 13:23:49,203 INFO sqlalchemy.engine.Engine [cached since 445.3s ago] {}
2024-04-04 13:23:49,299 INFO sqlalchemy.engine.Engine ROLLBACK
Training for station 1
2024-04-04 13:23:49,486 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-04 13:23:49,486 INFO sqlalchemy.engine.Engine SELECT `Availability`.`StationId`, `Availability`.`Status`, `Availability`.`MechanicalBikesAvailable`, `Availability`.`ElectricBikesAvailable`, `Availability`.`StandsAvailable`, `Availability`.`LastUpdated` 
FROM `Availability` 
WHERE `Availability`.`StationId` = %(StationId_1)s AND DATE(`Availability`.`LastUpdated`) >= %(DATE_1)s
2024-04-04 13:23:49,487 INFO sqlalchemy.en

/var/folders/k2/s9j0wvg542n0d4_0n5lwnp140000gn/T/ipykernel_91896/2593121471.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  availability_df['rounded_LastUpdated'] = pd.to_datetime(availability_df['LastUpdated']).dt.round('H')


AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:
# Drop every redundent columns
merged_data = merged_data.drop(columns=['Status','LastUpdated', 'DateTime','FeelsLike','Pop','UVI','WindGust','Rain1h','Snow1h',])
merged_data = merged_data.drop(columns=['last_update'])
merged_data = merged_data.drop(columns=['ForecastDate','time','minute_y','is_weekday'])

In [ ]:
merged_data

,StationId,MechanicalBikesAvailable,ElectricBikesAvailable,StandsAvailable,year,month,day,hour,minute_x,Humidity,Pressure,Temperature,WeatherId,WindSpeed
0,1,4,1,26,2024,3,7,14,43,72,1019,8.89,800,6.32
1,1,4,1,26,2024,3,7,14,43,72,1019,8.89,800,6.32
2,1,4,1,26,2024,3,7,14,43,72,1019,8.89,800,6.32
3,1,4,1,26,2024,3,7,14,43,74,1019,8.70,801,6.32
4,1,4,1,26,2024,3,7,14,43,75,1019,8.68,802,6.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,11,15,9,6,2024,3,7,14,42,72,1019,8.89,800,6.32
73,11,15,9,6,2024,3,7,14,42,74,1019,8.70,801,6.32
74,11,15,9,6,2024,3,7,14,42,75,1019,8.68,802,6.32
75,11,15,9,6,2024,3,7,14,42,75,1019,8.67,802,6.32


In [ ]:
merged_data.dtypes

StationId                     int64
MechanicalBikesAvailable      int64
ElectricBikesAvailable        int64
StandsAvailable               int64
year                          int64
month                         int64
day                           int64
hour                          int64
minute_x                      int64
Humidity                      int64
Pressure                      int64
Temperature                 float64
WeatherId                     int64
WindSpeed                   float64
dtype: object

In [ ]:
# Engineer is_busy_hours feature
merged_data['is_busy_hours'] = ((merged_data['hour'] >= 7) & (merged_data['hour'] <= 10)) | ((merged_data['hour'] >= 16) & (merged_data['hour'] <= 19)).astype(int)

In [ ]:
merged_data['cold_weather'] = (merged_data['Temperature'] < 5).astype(float)
merged_data['windy_weather'] = (merged_data['WindSpeed'] > 8).astype(float)

In [ ]:
merged_data

,StationId,MechanicalBikesAvailable,ElectricBikesAvailable,StandsAvailable,year,month,day,hour,minute_x,Humidity,Pressure,Temperature,WeatherId,WindSpeed,is_busy_hours,cold_weather,windy_weather
0,1,4,1,26,2024,3,7,14,43,72,1019,8.89,800,6.32,False,0.0,0.0
1,1,4,1,26,2024,3,7,14,43,72,1019,8.89,800,6.32,False,0.0,0.0
2,1,4,1,26,2024,3,7,14,43,72,1019,8.89,800,6.32,False,0.0,0.0
3,1,4,1,26,2024,3,7,14,43,74,1019,8.70,801,6.32,False,0.0,0.0
4,1,4,1,26,2024,3,7,14,43,75,1019,8.68,802,6.32,False,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,11,15,9,6,2024,3,7,14,42,72,1019,8.89,800,6.32,False,0.0,0.0
73,11,15,9,6,2024,3,7,14,42,74,1019,8.70,801,6.32,False,0.0,0.0
74,11,15,9,6,2024,3,7,14,42,75,1019,8.68,802,6.32,False,0.0,0.0
75,11,15,9,6,2024,3,7,14,42,75,1019,8.67,802,6.32,False,0.0,0.0
